# **Spark SQL Exercise Set – Product Orders Analytics**

**Dataset Theme: E-Commerce Orders**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('orderAnalytics').getOrCreate()
spark

1. Create a PySpark DataFrame with the following schema:

In [2]:
from pyspark.sql import Row

data = [
    Row(OrderID = 1, CustomerName = 'Karthika', Product = 'Saree', Category = 'Clothing', Quantity = 2, UnitPrice = 2000, OrderDate = '2025-07-20'),
    Row(OrderID = 2, CustomerName = 'Dharun', Product = 'Mobile', Category = 'Electronics', Quantity = 3, UnitPrice = 25000, OrderDate = '2023-01-20'),
    Row(OrderID = 3, CustomerName = 'Meena', Product = 'Dressing table', Category = 'Furniture', Quantity = 4, UnitPrice = 10000, OrderDate = '2024-09-25'),
    Row(OrderID = 4, CustomerName = 'Asha', Product = 'Kurta', Category = 'Clothing', Quantity = 7, UnitPrice = 500, OrderDate = '2025-02-19'),
    Row(OrderID = 5, CustomerName = 'Jayasree', Product = 'Bed', Category = 'Furniture', Quantity = 1, UnitPrice = 30000, OrderDate = '2022-07-22'),
    Row(OrderID = 6, CustomerName = 'Kalimthu', Product = 'Bluetooth', Category = 'Electronics', Quantity = 3, UnitPrice = 3000, OrderDate = '2025-03-06'),
    Row(OrderID = 7, CustomerName = 'Mohan', Product = 'Laptop', Category = 'Electronics', Quantity = 2, UnitPrice = 40000, OrderDate = '2025-05-15'),
    Row(OrderID = 8, CustomerName = 'Hareesh', Product = 'Comics', Category = 'Books', Quantity = 5, UnitPrice = 1500, OrderDate = '2023-10-10'),
    Row(OrderID = 9, CustomerName = 'Charan', Product = 'Science Fiction', Category = 'Books', Quantity = 8, UnitPrice = 2500, OrderDate = '2024-07-02'),
    Row(OrderID = 10, CustomerName = 'Siva', Product = 'Salwar', Category = 'Clothing', Quantity = 4, UnitPrice = 2700, OrderDate = '2025-02-28'),
    Row(OrderID = 11, CustomerName = 'Amutha', Product = 'Chair', Category = 'Furniture', Quantity = 9, UnitPrice = 1900, OrderDate = '2023-01-01'),
    Row(OrderID = 12, CustomerName = 'Ramya', Product = 'Television', Category = 'Electronics', Quantity = 1, UnitPrice = 55000, OrderDate = '2024-06-22'),
]

df = spark.createDataFrame(data)
df.show()

+-------+------------+---------------+-----------+--------+---------+----------+
|OrderID|CustomerName|        Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+---------------+-----------+--------+---------+----------+
|      1|    Karthika|          Saree|   Clothing|       2|     2000|2025-07-20|
|      2|      Dharun|         Mobile|Electronics|       3|    25000|2023-01-20|
|      3|       Meena| Dressing table|  Furniture|       4|    10000|2024-09-25|
|      4|        Asha|          Kurta|   Clothing|       7|      500|2025-02-19|
|      5|    Jayasree|            Bed|  Furniture|       1|    30000|2022-07-22|
|      6|    Kalimthu|      Bluetooth|Electronics|       3|     3000|2025-03-06|
|      7|       Mohan|         Laptop|Electronics|       2|    40000|2025-05-15|
|      8|     Hareesh|         Comics|      Books|       5|     1500|2023-10-10|
|      9|      Charan|Science Fiction|      Books|       8|     2500|2024-07-02|
|     10|        Siva|      

In [4]:
df.createOrReplaceTempView('orders_local')

**Part A: Local View – orders_local**

1. List all orders placed for "Electronics" with a Quantity of 2 or more.

In [8]:
spark.sql("SELECT * FROM orders_local WHERE Category = 'Electronics' AND Quantity >= 2").show()

+-------+------------+---------+-----------+--------+---------+----------+
|OrderID|CustomerName|  Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+---------+-----------+--------+---------+----------+
|      2|      Dharun|   Mobile|Electronics|       3|    25000|2023-01-20|
|      6|    Kalimthu|Bluetooth|Electronics|       3|     3000|2025-03-06|
|      7|       Mohan|   Laptop|Electronics|       2|    40000|2025-05-15|
+-------+------------+---------+-----------+--------+---------+----------+



2. Calculate TotalAmount (Quantity × UnitPrice) for each order.

In [9]:
spark.sql("SELECT OrderID, Quantity * UnitPrice AS TotalAmount FROM orders_local").show()

+-------+-----------+
|OrderID|TotalAmount|
+-------+-----------+
|      1|       4000|
|      2|      75000|
|      3|      40000|
|      4|       3500|
|      5|      30000|
|      6|       9000|
|      7|      80000|
|      8|       7500|
|      9|      20000|
|     10|      10800|
|     11|      17100|
|     12|      55000|
+-------+-----------+



3. Show the total number of orders per Category .

In [10]:
spark.sql("SELECT Category, COUNT(OrderID) FROM orders_local GROUP BY Category").show()

+-----------+--------------+
|   Category|count(OrderID)|
+-----------+--------------+
|Electronics|             4|
|   Clothing|             3|
|  Furniture|             3|
|      Books|             2|
+-----------+--------------+



4. List orders placed in "January 2023" only.

In [11]:
spark.sql("SELECT * FROM orders_local WHERE OrderDate LIKE '2023-01%'").show()

+-------+------------+-------+-----------+--------+---------+----------+
|OrderID|CustomerName|Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+-------+-----------+--------+---------+----------+
|      2|      Dharun| Mobile|Electronics|       3|    25000|2023-01-20|
|     11|      Amutha|  Chair|  Furniture|       9|     1900|2023-01-01|
+-------+------------+-------+-----------+--------+---------+----------+



5. Show the average UnitPrice per category.

In [14]:
spark.sql("SELECT Category, AVG(UnitPrice) AS avg_unitPrice FROM orders_local GROUP BY Category").show()

+-----------+------------------+
|   Category|     avg_unitPrice|
+-----------+------------------+
|Electronics|           30750.0|
|   Clothing|1733.3333333333333|
|  Furniture|13966.666666666666|
|      Books|            2000.0|
+-----------+------------------+



6. Find the order with the highest total amount.

In [17]:
spark.sql("SELECT OrderID, Quantity * UnitPrice AS TotAmount FROM orders_local ORDER BY TotAmount DESC LIMIT 1").show()

+-------+---------+
|OrderID|TotAmount|
+-------+---------+
|      7|    80000|
+-------+---------+



7. Drop the local view and try querying it again.

In [19]:
spark.sql("DROP VIEW orders_local")

spark.sql("SELECT * FROM orders_local").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`orders_local` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

**Part B: Global View – orders_global**

1. Display all "Furniture" orders with TotalAmount above 10,000.

In [20]:
new_spark = SparkSession.builder.appName('orderAnalytics1').getOrCreate()

df.createOrReplaceGlobalTempView('orders_global')

new_spark.sql("SELECT * FROM global_temp.orders_global WHERE Category = 'Furniture' AND Quantity * UnitPrice > 10000").show()

+-------+------------+--------------+---------+--------+---------+----------+
|OrderID|CustomerName|       Product| Category|Quantity|UnitPrice| OrderDate|
+-------+------------+--------------+---------+--------+---------+----------+
|      3|       Meena|Dressing table|Furniture|       4|    10000|2024-09-25|
|      5|    Jayasree|           Bed|Furniture|       1|    30000|2022-07-22|
|     11|      Amutha|         Chair|Furniture|       9|     1900|2023-01-01|
+-------+------------+--------------+---------+--------+---------+----------+



2. Create a column called DiscountFlag :

Mark "Yes" if Quantity > 3
Otherwise "No"

In [24]:
new_spark.sql("SELECT *, CASE WHEN Quantity > 3 THEN 'Yes' ELSE 'No' END AS DiscountFlag FROM global_temp.orders_global").show()

+-------+------------+---------------+-----------+--------+---------+----------+------------+
|OrderID|CustomerName|        Product|   Category|Quantity|UnitPrice| OrderDate|DiscountFlag|
+-------+------------+---------------+-----------+--------+---------+----------+------------+
|      1|    Karthika|          Saree|   Clothing|       2|     2000|2025-07-20|          No|
|      2|      Dharun|         Mobile|Electronics|       3|    25000|2023-01-20|          No|
|      3|       Meena| Dressing table|  Furniture|       4|    10000|2024-09-25|         Yes|
|      4|        Asha|          Kurta|   Clothing|       7|      500|2025-02-19|         Yes|
|      5|    Jayasree|            Bed|  Furniture|       1|    30000|2022-07-22|          No|
|      6|    Kalimthu|      Bluetooth|Electronics|       3|     3000|2025-03-06|          No|
|      7|       Mohan|         Laptop|Electronics|       2|    40000|2025-05-15|          No|
|      8|     Hareesh|         Comics|      Books|       5| 

3. List customers who ordered more than 1 product type (Hint: use GROUP BY and HAVING).

In [27]:
new_spark.sql("SELECT CustomerName, COUNT(Product) FROM global_temp.orders_global GROUP BY CustomerName HAVING COUNT(Product) > 1").show()

+------------+--------------+
|CustomerName|count(Product)|
+------------+--------------+
+------------+--------------+



4. Count number of orders per month across the dataset.

In [29]:
new_spark.sql("SELECT COUNT(OrderID) AS no_of_orders, MONTH(OrderDate) AS Month FROM global_temp.orders_global GROUP BY Month").show()

+------------+-----+
|no_of_orders|Month|
+------------+-----+
|           2|    1|
|           1|    3|
|           1|    9|
|           3|    7|
|           2|    2|
|           1|    6|
|           1|    5|
|           1|   10|
+------------+-----+



5. Rank all products by total quantity sold across all orders using a window function.

In [30]:
new_spark.sql("SELECT Product, SUM(Quantity) AS TotalQuantity, RANK() OVER (ORDER BY SUM(Quantity) DESC) AS Rank FROM global_temp.orders_global GROUP BY Product").show()

+---------------+-------------+----+
|        Product|TotalQuantity|Rank|
+---------------+-------------+----+
|          Chair|            9|   1|
|Science Fiction|            8|   2|
|          Kurta|            7|   3|
|         Comics|            5|   4|
| Dressing table|            4|   5|
|         Salwar|            4|   5|
|      Bluetooth|            3|   7|
|         Mobile|            3|   7|
|          Saree|            2|   9|
|         Laptop|            2|   9|
|            Bed|            1|  11|
|     Television|            1|  11|
+---------------+-------------+----+



6. Run a query using a new SparkSession and the global view.

In [31]:
spark1 = SparkSession.builder.appName('orderAnalytics2').getOrCreate()

spark1.sql("SELECT * FROM global_temp.orders_global").show()

+-------+------------+---------------+-----------+--------+---------+----------+
|OrderID|CustomerName|        Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+---------------+-----------+--------+---------+----------+
|      1|    Karthika|          Saree|   Clothing|       2|     2000|2025-07-20|
|      2|      Dharun|         Mobile|Electronics|       3|    25000|2023-01-20|
|      3|       Meena| Dressing table|  Furniture|       4|    10000|2024-09-25|
|      4|        Asha|          Kurta|   Clothing|       7|      500|2025-02-19|
|      5|    Jayasree|            Bed|  Furniture|       1|    30000|2022-07-22|
|      6|    Kalimthu|      Bluetooth|Electronics|       3|     3000|2025-03-06|
|      7|       Mohan|         Laptop|Electronics|       2|    40000|2025-05-15|
|      8|     Hareesh|         Comics|      Books|       5|     1500|2023-10-10|
|      9|      Charan|Science Fiction|      Books|       8|     2500|2024-07-02|
|     10|        Siva|      

**Bonus Challenges**

1. Save a filtered subset (only "Books" category) as a new global temp view.

In [32]:
df.createOrReplaceGlobalTempView('books_global')

new_spark.sql("SELECT * FROM global_temp.books_global WHERE Category = 'Books'").show()

+-------+------------+---------------+--------+--------+---------+----------+
|OrderID|CustomerName|        Product|Category|Quantity|UnitPrice| OrderDate|
+-------+------------+---------------+--------+--------+---------+----------+
|      8|     Hareesh|         Comics|   Books|       5|     1500|2023-10-10|
|      9|      Charan|Science Fiction|   Books|       8|     2500|2024-07-02|
+-------+------------+---------------+--------+--------+---------+----------+



2. Find the most purchased product per category.

In [33]:
new_spark.sql("SELECT Category, Product, COUNT(OrderID) AS PurchaseCount FROM global_temp.orders_global GROUP BY Category, Product ORDER BY PurchaseCount DESC").show()

+-----------+---------------+-------------+
|   Category|        Product|PurchaseCount|
+-----------+---------------+-------------+
|  Furniture| Dressing table|            1|
|Electronics|      Bluetooth|            1|
|  Furniture|            Bed|            1|
|Electronics|         Mobile|            1|
|   Clothing|          Saree|            1|
|   Clothing|          Kurta|            1|
|Electronics|     Television|            1|
|      Books|         Comics|            1|
|      Books|Science Fiction|            1|
|  Furniture|          Chair|            1|
|Electronics|         Laptop|            1|
|   Clothing|         Salwar|            1|
+-----------+---------------+-------------+



3. Create a view that excludes all "Clothing" orders and call it
"filtered_orders" .

In [34]:
df.createOrReplaceGlobalTempView('filtered_orders')

new_spark.sql("SELECT * FROM global_temp.filtered_orders WHERE Category != 'Clothing'").show()

+-------+------------+---------------+-----------+--------+---------+----------+
|OrderID|CustomerName|        Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+---------------+-----------+--------+---------+----------+
|      2|      Dharun|         Mobile|Electronics|       3|    25000|2023-01-20|
|      3|       Meena| Dressing table|  Furniture|       4|    10000|2024-09-25|
|      5|    Jayasree|            Bed|  Furniture|       1|    30000|2022-07-22|
|      6|    Kalimthu|      Bluetooth|Electronics|       3|     3000|2025-03-06|
|      7|       Mohan|         Laptop|Electronics|       2|    40000|2025-05-15|
|      8|     Hareesh|         Comics|      Books|       5|     1500|2023-10-10|
|      9|      Charan|Science Fiction|      Books|       8|     2500|2024-07-02|
|     11|      Amutha|          Chair|  Furniture|       9|     1900|2023-01-01|
|     12|       Ramya|     Television|Electronics|       1|    55000|2024-06-22|
+-------+------------+------